In [1]:
import os
import numpy as np
import pandas as pd
from pprint import pprint

def open_dataset(file_name):
    return pd.read_csv(filepath_or_buffer=file_name, delimiter="\t", encoding="utf-8", header=0)

# itineraries = open_dataset('data_sncf/timetables.csv')
itineraries = open_dataset('data_sncf/test.csv')
itineraries = itineraries.sort_values(by=['trip_id', 'trajet', 'duree'], ascending=[True, True, True])

# refacto du dataframe
trajet_splited = itineraries['trajet'].str.split(' - ')

itineraries['start'] = trajet_splited.str[0]
itineraries['end'] = trajet_splited.str[1]

itineraries.drop('trajet', axis=1, inplace=True)

itineraries

,trip_id,duree,start,end
0,OCESN003100F140147152,82,Paris,Marseille
1,OCESN003190F040047309,27,Paris,Lyon
2,OCESN003198F030037315,34,Lyon,Marseille
3,OCESN003300F030037323,11,Paris,Annecy
4,OCESN003313F380387526,13,Annecy,Grenoble
5,OCESN003371F080087908,23,Grenoble,Marseille


In [2]:
findTravel = itineraries.loc[itineraries['trip_id'] == "OCESN014033F0900915597"]
print(findTravel)

Empty DataFrame
Columns: [trip_id, duree, start, end]
Index: []


In [3]:
locsStart = itineraries['start'].tolist()
locsEnd = itineraries['end'].tolist()
locs = list(dict.fromkeys(locsStart + locsEnd))
# len(locs)
locs



['Paris', 'Lyon', 'Annecy', 'Grenoble', 'Marseille']

In [4]:
def get_itineraries(fromLoc):
    return itineraries.loc[itineraries['start'] == fromLoc]


In [5]:
# update matrix from itineraries
def updateMatrixNodes(matrix, itineraries):
    # print(itineraries)
    
    for index, row in itineraries.iterrows():
        
        # if initerary is not yet processed
        if matrix[row['end']]['processed'] == False:
        
            # print(matrix[row['end']]['distanceFromSource'])
            oldValue = matrix[row['end']]['distanceFromSource']
            newValue = matrix[row['start']]['distanceFromSource'] + row['duree']
            
            if oldValue == 'infinite':
                matrix[row['end']]['distanceFromSource'] = newValue
                matrix[row['end']]['fromLoc'] = row['start']
            else:   
                if newValue < oldValue:
                    matrix[row['end']]['distanceFromSource'] = newValue
                    matrix[row['end']]['fromLoc'] = row['start']
        
        
    print("-----------------------------------------------------------------")
    print("---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------")
    print("-----------------------------------------------------------------")
    print(" ")
    
    return matrix

In [6]:
# Find the minimal distance who are not processed and who is not infinite
def findMiniDistanceFromSource(matrix):
    
    toProcess = {}
    
    for index, row in matrix.items():
        if row['processed'] == False and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row
            
    print("-------------------- Les itineraires pas encore traités ---------------------")
    print(" ")
    [print(key,':',value) for key, value in toProcess.items()]
    print(" ")
    
    miniValue = 9999999999
    location = ""
    
    for index, row in toProcess.items():
    
        if row['distanceFromSource'] < miniValue:
            miniValue = row['distanceFromSource']
            location = index
            
       
    if location != "":    
        print("-------------------- La distance la plus courte pas encore traitée ---------------------")
        print(" ")
        print("Etape : ", location, " - Distance depuis la source : ", miniValue)
        print(" ")
    else:
        print("------------------------------------------------------------------------")
        print("-------------------- Plus de trajets lié à traiter ---------------------")
        print("------------------------------------------------------------------------")
        print(" ")
        
    return location
    
                
    

In [7]:
def get_best_itinerary(matrix, source, destination):
    toProcess = {}
    distance = ""
    fromLoc = ""
    itineraries = []
    
    for index, row in matrix.items():
        if row['processed'] == True and row['distanceFromSource'] != 'infinite':
            toProcess[index] = row
            
    # print(toProcess)
    if destination in toProcess.keys():
        print(toProcess[destination])
        distance = toProcess[destination]["distanceFromSource"]
        fromLoc = toProcess[destination]["fromLoc"]
        itineraries.insert(0, destination)
        
        while fromLoc != source:
            itineraries.insert(0, fromLoc)
            fromLoc = toProcess[fromLoc]["fromLoc"]
            
        itineraries.insert(0, source)
        
        return {
            'itineraries': itineraries,
            'distance': distance
        }
    
    else:
        print("-----------------------------------------------------------------")
        print("-------------------- Ce trajet n'existe pas ---------------------")
        print("-----------------------------------------------------------------")
        print("")
            
    # for index, row in toProcess.items():
    #     if row['processed'] == True and row['distanceFromSource'] != 'infinite':
    #         toProcess[index] = row
            
    return toProcess
    

In [8]:

def dijsktra(df, source, destination):
    matrixLoc = {}
    
    locsStart = df['start'].tolist()
    locsEnd = df['end'].tolist()
    locs = list(dict.fromkeys(locsStart + locsEnd))
    
    # If multi start & end locations possible with this location name, get the first result
    locStart = list(filter(lambda k: source in k, locs))[0]
    locEnd = list(filter(lambda k: destination in k, locs))[0]
    
    print("Trajet : ", locStart, " - ", locEnd)
    print(" ")
    
    # initialize matrix
    for loc in locs:
        matrixLoc[loc] = {
            'distanceFromSource': 'infinite',
            'fromLoc': '',
            'processed': False,
        }
        
    # initialize matrix source values
    matrixLoc[locStart]['distanceFromSource'] = 0
    matrixLoc[locStart]['fromLoc'] = locStart
    matrixLoc[locStart]['processed'] = True
    
    nextLocation = locStart
    
    while nextLocation != "":
        # print(get_itineraries(nextLocation))
        if not get_itineraries(nextLocation).empty:
            matrixLoc = updateMatrixNodes(matrixLoc, get_itineraries(nextLocation))
        else:
            print("Aucun itinéraire en partant de ", nextLocation)
            print(" ")
            
        nextLocation = findMiniDistanceFromSource(matrixLoc)
        # print(nextLocation)
        if nextLocation != "":
            matrixLoc[nextLocation]['processed'] = True
    
    return get_best_itinerary(matrixLoc, locStart, locEnd)
    
    

In [9]:
locStart = "Paris"
locEnd = "Marseille"

dijsktra(itineraries, locStart, locEnd)


Trajet :  Paris  -  Marseille
 
-----------------------------------------------------------------
---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------
-----------------------------------------------------------------
 
-------------------- Les itineraires pas encore traités ---------------------
 
Lyon : {'distanceFromSource': 27, 'fromLoc': 'Paris', 'processed': False}
Annecy : {'distanceFromSource': 11, 'fromLoc': 'Paris', 'processed': False}
Marseille : {'distanceFromSource': 82, 'fromLoc': 'Paris', 'processed': False}
 
-------------------- La distance la plus courte pas encore traitée ---------------------
 
Etape :  Annecy  - Distance depuis la source :  11
 
-----------------------------------------------------------------
---------- MISE A JOUR DES DISTANCES DEPUIS LA SOURCE -----------
-----------------------------------------------------------------
 
-------------------- Les itineraires pas encore traités ---------------------
 
Lyon : {'distanceFromSource': 27,

{'itineraries': ['Paris', 'Annecy', 'Grenoble', 'Marseille'], 'distance': 47}